In [9]:
import json
import psycopg2
import pandas as pd

#this is a .json with my credentials, see also update-department-boundaries...
#for a more convenient approach using your pg_conf file
with open('../config.json') as f:
    conf = json.load(f)

In [15]:
conn_str = "host={host} dbname={database} user={user} password={passw} port={port}".format(**conf)

In [16]:
conn = psycopg2.connect(conn_str)

In [17]:
bigquery = """select r.department_id, round(s.yravg,2) as yravg_fires, round(r.risk_model_fires::numeric,2) as predicted_fires,fd.name from 
firestation_firedepartmentriskmodels r
	inner join (select fire_department_id, avg(count) as yravg from firestation_nfirsstatistic where metric='residential_structure_fires'
and count is not null and year >= 2010 and level = 0 group by fire_department_id) s
	on r.department_id = s.fire_department_id
	inner join firestation_firedepartment fd on r.department_id = fd.id
where r.level = 0 order by yravg desc
"""

In [19]:
df = pd.read_sql(bigquery,con=conn)

In [47]:
df.head()

,department_id,yravg_fires,predicted_fires,name,residuals
0,77629,3665.00,3517.99,City of Detroit Fire Department,147.01
1,81472,3026.29,32999.49,Fire Department City of New York (FDNY),-29973.20
2,91907,2575.83,952.82,Philadelphia Fire Department,1623.01
3,87256,1853.86,3039.74,Los Angeles Fire Department,-1185.88
4,77379,1450.86,5805.21,Chicago Fire Department,-4354.35


In [32]:
fn = lambda row: row.yravg_fires-row.predicted_fires
col = df.apply(fn,axis=1)
df = df.assign(residuals=col.values)

In [58]:
fn = lambda row: (row.yravg_fires-row.predicted_fires)/(row.yravg_fires)
col = df.apply(fn,axis=1)
df = df.assign(normresid=col.values) #percentage of avg our model is off (error rate in a sense)

In [48]:
# 90% of fire departments reporting more than 100 incidents have predicted values within about 600 incidents of the true value.
df.residuals.describe()

count     2578.000000
mean       -37.937052
std        604.690558
min     -29973.200000
25%        -22.060000
50%         -9.265000
75%         -0.135000
max       1623.010000
Name: residuals, dtype: float64

In [62]:
df.normresid.describe()

count    2578.000000
mean       -0.845660
std         7.705099
min      -370.576779
25%        -1.057041
50%        -0.450421
75%        -0.006760
max         1.000000
Name: normresid, dtype: float64

In [64]:
df[(df.residuals < 1000) & (df.residuals > -1000) & (df.yravg_fires > 1000)].head()

,department_id,yravg_fires,predicted_fires,name,residuals,normresid
0,77629,3665.00,3517.99,City of Detroit Fire Department,147.01,0.040112
5,84578,1021.14,1709.10,Houston Fire Department,-687.96,-0.673718
6,91934,1017.57,692.41,Phoenix Fire Department,325.16,0.319546


In [65]:
df[(abs(df.normresid) < 0.5)].head()

,department_id,yravg_fires,predicted_fires,name,residuals,normresid
0,77629,3665.00,3517.99,City of Detroit Fire Department,147.01,0.040112
6,91934,1017.57,692.41,Phoenix Fire Department,325.16,0.319546
7,88821,968.14,1053.54,Milwaukee Fire Department,-85.40,-0.088210
8,88403,835.71,1210.36,Memphis Division of Fire Services,-374.65,-0.448301
10,78139,761.29,909.69,Cleveland Division of Fire,-148.40,-0.194932


In [66]:
df[df.yravg_fires > 100]

,department_id,yravg_fires,predicted_fires,name,residuals,normresid
0,77629,3665.00,3517.99,City of Detroit Fire Department,147.01,0.040112
1,81472,3026.29,32999.49,Fire Department City of New York (FDNY),-29973.20,-9.904272
2,91907,2575.83,952.82,Philadelphia Fire Department,1623.01,0.630092
3,87256,1853.86,3039.74,Los Angeles Fire Department,-1185.88,-0.639682
4,77379,1450.86,5805.21,Chicago Fire Department,-4354.35,-3.001220
5,84578,1021.14,1709.10,Houston Fire Department,-687.96,-0.673718
6,91934,1017.57,692.41,Phoenix Fire Department,325.16,0.319546
7,88821,968.14,1053.54,Milwaukee Fire Department,-85.40,-0.088210
8,88403,835.71,1210.36,Memphis Division of Fire Services,-374.65,-0.448301
9,94264,824.00,1914.11,San Francisco Fire Department,-1090.11,-1.322949
